In [11]:
# Import necessary libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [12]:
# Load dataset
dataset = pd.read_csv('Mumbai_LSTM_2.5.csv')
data = dataset.values
data = data.astype('float32')

In [13]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [14]:
# Split data into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [15]:
# Create function to create LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(60, input_shape=(1, 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [16]:
# Reshape input data to be 3-dimensional
X_train = np.reshape(train, (train.shape[0], 1, train.shape[1]))
X_test = np.reshape(test, (test.shape[0], 1, test.shape[1]))

In [17]:
# Create and fit the LSTM model
model = create_model()
model.fit(X_train, train, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
927/927 - 9s - loss: 0.0206 - 9s/epoch - 10ms/step
Epoch 2/100
927/927 - 4s - loss: 0.0012 - 4s/epoch - 5ms/step
Epoch 3/100
927/927 - 4s - loss: 0.0010 - 4s/epoch - 5ms/step
Epoch 4/100
927/927 - 4s - loss: 7.3402e-04 - 4s/epoch - 5ms/step
Epoch 5/100
927/927 - 4s - loss: 9.1534e-04 - 4s/epoch - 5ms/step
Epoch 6/100
927/927 - 4s - loss: 6.9481e-04 - 4s/epoch - 5ms/step
Epoch 7/100
927/927 - 4s - loss: 5.5786e-04 - 4s/epoch - 5ms/step
Epoch 8/100
927/927 - 4s - loss: 6.5864e-04 - 4s/epoch - 5ms/step
Epoch 9/100
927/927 - 4s - loss: 6.0914e-04 - 4s/epoch - 5ms/step
Epoch 10/100
927/927 - 4s - loss: 5.3619e-04 - 4s/epoch - 5ms/step
Epoch 11/100
927/927 - 4s - loss: 5.0508e-04 - 4s/epoch - 5ms/step
Epoch 12/100
927/927 - 4s - loss: 5.1043e-04 - 4s/epoch - 5ms/step
Epoch 13/100
927/927 - 4s - loss: 3.9953e-04 - 4s/epoch - 5ms/step
Epoch 14/100
927/927 - 4s - loss: 3.8533e-04 - 4s/epoch - 5ms/step
Epoch 15/100
927/927 - 4s - loss: 4.4903e-04 - 4s/epoch - 4ms/step
Epoch 16/100
92

In [18]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

15/15 [==============================] - 0s 5ms/step


In [19]:
train_predict = scaler.inverse_transform(train_predict)
train = scaler.inverse_transform(train)
test_predict = scaler.inverse_transform(test_predict)
test = scaler.inverse_transform(test)

In [20]:
"""train_rmse = np.sqrt(np.mean(np.power((train - train_predict), 2)))
test_rmse = np.sqrt(np.mean(np.power((test - test_predict), 2)))
print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)"""
from sklearn.metrics import r2_score
import math
from sklearn.metrics import mean_squared_error
train_rmse = np.sqrt(mean_squared_error(train, train_predict))
test_rmse = np.sqrt(mean_squared_error(test, test_predict))
print('R2 Score_train:', r2_score(train, train_predict))
print('R2 Score_test:', r2_score(test, test_predict))

print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)

R2 Score_train: 0.9997482722503827
R2 Score_test: 0.9996684802686403
Train RMSE: 0.53
Test RMSE: 0.50


In [21]:
df=pd.DataFrame({'Actual':train.tolist(),'Predicted from lstm':train_predict.tolist()})
df.to_excel('lstm_2.5.xlsx')
df

,Actual,Predicted from lstm
0,[42.59090042114258],[42.026710510253906]
1,[42.59090042114258],[42.026710510253906]
2,[42.59090042114258],[42.026710510253906]
3,[42.59090042114258],[42.026710510253906]
4,[42.59090042114258],[42.026710510253906]
...,...,...
922,[108.7300033569336],[109.09931182861328]
923,[57.650001525878906],[57.32402038574219]
924,[64.30999755859375],[64.10700225830078]
925,[100.80000305175781],[101.14124298095703]


In [22]:
df=pd.DataFrame({'Actual':test.tolist(),'Predicted from lstm':test_predict.tolist()})
df.to_excel('lstm_2.5_test.xlsx')
df

,Actual,Predicted from lstm
0,[96.51000213623047],[96.81705474853516]
1,[59.09000015258789],[58.79013442993164]
2,[87.27999877929688],[87.4769287109375]
3,[120.0],[120.31486511230469]
4,[88.94999694824219],[89.17005157470703]
...,...,...
453,[33.689998626708984],[33.034156799316406]
454,[39.560001373291016],[38.95936584472656]
455,[36.15999984741211],[35.52472686767578]
456,[23.6299991607666],[22.937284469604492]
